In [1]:
from input_output import *
from functools import reduce
import altair as alt
print("Available datasets:\n\t" + ',\n\t'.join(os.listdir(sims_root())))

In [2]:
# Edit this cell to specify which datasets to compare
# dataset_names = ["WTSolidCube_7Nov22", "AlloSolidCube_X1_7Nov22", "AlloSolidCube_X3_Singlet_7Nov22", "AlloSolidCube_X5_1_7Nov22", "AlloSolidCube_X5_4_7Nov22"]
# dataset_names = ["WTHollowCube_7Nov22", "AlloHollowCubeMin_X1_9Nov22", "AlloHollowCube_X1_7Nov22"]
dataset_names = ["Allosteric_Pyramid_Full_Fall22", "Allosteric_Pyramid_Partial_Fall22", "NoAlloPyramid_14Oct22"]
### Readability ### 
# make sure the order is the same as dataset_names!!!!
dataset_names_readable = ["Full Allosteric", "Partial Allosteric", "Non-Allosteric"]
group_name = "Pyramid"
target_name = "pyramid"

In [4]:
assert(len(dataset_names) > 0)
datasets = [choose_results(dsname) for dsname in dataset_names]

In [61]:
datasets_data = []
for dataset, dsname, dsnamehuman in zip(datasets, dataset_names, dataset_names_readable):
    print(f"Analyzing {dsname}")
    data = dataset.get_all_sizes()
    # data = dataset.get_flat_yields(target_name, cutoff, overreach=overreach)
    data['dsname'] = dsnamehuman
    datasets_data.append(data)
data = pd.concat(datasets_data, axis='index').reset_index()

In [62]:
data["nt"].unique()

In [63]:
tmax = 8 * get_sample_every() # pretty arbitrary, subject to review

In [64]:
data = data[data["timepoint"] <= tmax]

In [65]:
data["time"] = data["timepoint"] * OXDNA_PRINT_CLUSTERS_EVERY / 1e6 # convert to megasteps

In [66]:
df

In [71]:
nt = 2

df = data[data["nt"] == nt]

ymax = data["total_size"].max()

chart = alt.Chart(df).mark_line().encode(
    x=alt.X('time:Q', title="Time (megasteps)", axis=alt.Axis(tickCount=5)),
    y=alt.Y('total_size:Q', title="Total Assembly Size", scale=alt.Scale(domain=(0,ymax)), aggregate="mean"),
    color=alt.Color('dsname:O', scale=alt.Scale(scheme="accent"), title="Rule"),
    strokeDash=alt.Stroke('temp', title="Temperature")
)

chart = chart.configure_axis(
    labelFontSize=18,
    titleFontSize=20
).configure_title(fontSize=24)
chart.configure_legend(
        labelFontSize=18,
    titleFontSize=20
)